In [25]:

import numpy as np

import torch.nn as nn
import torch.optim as optim

In [26]:
import torch
import torchvision
from torchvision import transforms as transforms



In [118]:
BATCH_SIZE = 20

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=train_transform)
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True) # 10000个sample ( batch 500 ,batch_num 20)

Files already downloaded and verified


In [119]:
nodes = 5
INPUT_DIM = 3*32*32
NUM_CLASSES = 10
BATCH_SIZE= 20

In [120]:


class AlexNet(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, num=1):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
      )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x

In [134]:
_data_agg = np.zeros([BATCH_SIZE*nodes,INPUT_DIM],dtype=np.float32)
_data_perturbed_agg = np.zeros([BATCH_SIZE*nodes,INPUT_DIM],dtype=np.float32)
_labels_agg = np.zeros([BATCH_SIZE*nodes],dtype=np.int64)

In [135]:
models ={}

for i in range(nodes):
    model_name = f"{i}"  # 给模型指定名称，例如 "model_1", "model_2", ...
    model = AlexNet(num=i)
    models[model_name] = model

models.keys()


dict_keys(['0', '1', '2', '3', '4'])

In [136]:
theta = nn.Parameter(torch.zeros(nodes, INPUT_DIM))
theta_1 = nn.Parameter(torch.ones(nodes, INPUT_DIM))
maxVar = nn.Parameter(torch.zeros(nodes, INPUT_DIM))
maxVar_1 = nn.Parameter(torch.zeros(nodes, INPUT_DIM))

nn_params = list(models[str(i)].parameters()) + [theta] + [theta_1]
max_params = [maxVar]+[maxVar_1]

Classifier_train_op = optim.Adam([ {'params': nn_params, 'lr': 1e-3}])
Max_train_op = optim.Adam([{'params': max_params, 'lr': 1e-4}])

In [137]:
criterion = nn.CrossEntropyLoss()
LAMBDA_0 = 10.0
LAMBDA_1 = 1.0
adv_stepsize = 2.0


In [142]:
noise_std = 5.0
perturbation_add_train = noise_std*np.random.normal(size=[nodes,INPUT_DIM])
matrix_mult_train = (noise_std/(np.sqrt(INPUT_DIM)))*np.random.normal(size=[nodes,INPUT_DIM,INPUT_DIM])

In [1]:

for batch_num, (data, target) in enumerate(train_loader):
    Max_train_op.zero_grad()
    Classifier_train_op.zero_grad()

    for k in range(nodes):
        _data = np.array(data) # todo : npararry
        _data_agg[k*BATCH_SIZE:(k+1)*BATCH_SIZE,:] += np.matmul(_data_agg[k*BATCH_SIZE:(k+1)*BATCH_SIZE,:], np.squeeze(matrix_mult_train[k,:,:]))
        _data_agg[k*BATCH_SIZE:(k+1)*BATCH_SIZE,:] +=  perturbation_add_train[k,:]
        _labels_agg[k*BATCH_SIZE:(k+1)*BATCH_SIZE] = np.array(target) # todo : npararry
    # print(_data_agg)
    _data = np.int32(_data_agg)
    _labels = _labels_agg

    real_data = torch.tensor(_data, dtype = torch.float32)
    label = torch.tensor( _labels, dtype = torch.int64)

    data_perturbed_list = []
    data_perturbed_list_max = []
    NN_out_perturbed_list = []

    for i in range(nodes):
        # data_perturbed_list.append(torch.mul(real_data[i * BATCH_SIZE:(i + 1) * BATCH_SIZE, :], theta_1[i, :]) + theta[i, :]) # (10,3072)
        data_perturbed_list.append(torch.mul(real_data[i * BATCH_SIZE:(i + 1) * BATCH_SIZE, :], theta_1[i, :]) + theta[i, :])

    data_perturbed = torch.stack(data_perturbed_list) # ([100, 10, 3072])
    data_perturbed_max = data_perturbed.clone().mean(dim=1)
    data_perturbed_pow_2_max = (data_perturbed ** 2).clone().mean(dim=1) # (100,3072)

    for i in range(nodes):
        NN_out_perturbed_list.append(models[str(i)](data_perturbed[i, :, :].reshape(BATCH_SIZE,3,32,32 )))

    NN_out_perturbed = torch.stack(NN_out_perturbed_list)
    NN_out_perturbed = NN_out_perturbed.view(BATCH_SIZE * nodes, 10) # [1000, 10]

    train_acc = (NN_out_perturbed.argmax(dim=1) == label).float().mean()
    train_loss = criterion(NN_out_perturbed, label)

    # print('NN_out_perturbed.argmax(dim=1) ',NN_out_perturbed.argmax(dim=1) )

    max_loss = torch.sum(data_perturbed_max * (maxVar.clone() - maxVar.clone().mean(dim=0)))
    max_loss_1 = torch.sum(data_perturbed_pow_2_max * (maxVar_1.clone() - maxVar_1.clone().mean(dim=0)))

    train_loss_2 = train_loss - LAMBDA_0 * torch.sum(maxVar.clone() ** 2) - LAMBDA_1 * torch.sum(maxVar_1.clone() ** 2) + adv_stepsize * (max_loss + max_loss_1)


    print('train_acc {}, train_loss_2 {}'.format(train_acc, train_loss_2))


    Classifier_train_loss =  train_loss_2.clone()
    Max_train_op_loss = - train_loss_2.clone()


    Max_train_op_loss.backward(retain_graph=True)
    Max_train_op.step()

    Classifier_train_loss.backward()
    Classifier_train_op.step()